# Assignment 3: Optimization II

University of California Berkeley

ME C231A, EE C220B, Experiential Advanced Control I

***

These notes were developed by Roya Firoozi and Francesco Borrelli at UC Berkeley. They are protected by U.S. copyright law and by University policy (https://copyright.universityofcalifornia.edu/resources/ownership-course-materials.html).

If you are enrolled in ME C231A/EE C220B you may take notes and make copies of course materials for your own use. You may also share those materials with another student who is registered and enrolled in this course, and with DSP.

You may not reproduce, distribute or display (post/upload) (Links to an external site.) lecture notes or recordings or course materials in any other way — whether or not a fee is charged — without my express written consent. You also may not allow others to do so. If you do so, you may be subject to student conduct proceedings under the Links to an external site.Berkeley Code of Student Conduct, including Sections 102.23 and 102.25.

***

In [ ]:
# Please run this cell only if you are using Google Colab. 

# install required dependencies
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
  !pip install -q pyomo
  !apt-get install -y -qq glpk-utils
  !apt-get install -y -qq coinor-cbc
  !wget -N -q "https://ampl.com/dl/open/ipopt/ipopt-linux64.zip"
  !unzip -o -q ipopt-linux64

Note: You should now be familiar with $\texttt{cvxopt}$ to solve linear and quadratic programs. For certain problems in this assignment you will need to use a nonlinear solver. You may specify $\texttt{IPOPT}$ as a solver in $\texttt{pyomo}$ which solves constrained, nonlinear programs. 

***

# <font color=blue> 1. Linear and Quadratic Programming: </font>

The following problems include some of the examples from homework 2. This time, use $\texttt{Pyomo}$ to solve them. Please submit your solutions as individual functions for each of the 4 parts. 

These functions should have no inputs and 4 outputs. The first two outputs are logical values indicating the feasibility and  boundedness, where a [1,1] stands for a feasible problem and bounded solution. The third output is the value of the optimizer, which should be an $N\times 1$ vector, where $N$ is the dimension of decision variable. If the problem is infeasible or unbounded, the function should return an empty array here (i.e. $\texttt{zOpt = []}$). The fourth output is the optimal value of the cost function. If the problem is infeasible, the function should return an empty array here. If the problem is unbounded return $\texttt{+inf}$ or $\texttt{-inf}$ here. You can call the function $\texttt{check_solver_status}$ to determine if the problem is feasible/bounded or not.  

In [ ]:
import pyomo.environ as pyo
import numpy as np
def check_solver_status(model, results):
    from pyomo.opt import SolverStatus, TerminationCondition
    if (results.solver.status == SolverStatus.ok) and (results.solver.termination_condition == TerminationCondition.optimal):
        print('========================================================================================')
        print('================ Problem is feasible and the optimal solution is found ==================')
        # print('z1 optimal=', pyo.value(model.z[1]))
        # print('z2 optimal=', pyo.value(model.z[2]))
        print('z1 optimal=', pyo.value(model.z1))
        print('z2 optimal=', pyo.value(model.z2))       
        print('optimal value=', pyo.value(model.obj))
        print('========================================================================================')
        bound = True
        feas = True
        # zOpt = np.array([pyo.value(model.z[1]), pyo.value(model.z[2])])
        zOpt = np.array([pyo.value(model.z1), pyo.value(model.z2)])

        JOpt = pyo.value(model.obj)
    elif (results.solver.termination_condition == TerminationCondition.infeasible):
        print('========================================================')
        print('================ Problem is infeasible ==================')
        print('========================================================')
        feas = False
        zOpt = []
        JOpt = []
        if (results.solver.termination_condition == TerminationCondition.unbounded):
            print('================ Problem is unbounded ==================')
            bound = False
        else:
            bound = True
        
    else:
        if (results.solver.termination_condition == TerminationCondition.unbounded):
            print('================ Problem is unbounded ==================')
            bound = False
            feas = True
            zOpt = []
            JOpt = np.inf
        else:
            bound = True
            feas = True
            zOpt = []
            JOpt = np.inf
            
    return feas, bound, zOpt, JOpt

$\textbf{Part (a)}$

\begin{align}
\min_{z_1,z_2}~ &  -5z_1 -7z_2  \nonumber\\
\text{s.t. } & -3z_1 +2z_2 \leq 30  \nonumber\\
& -2z_1 + z_2 \leq 12  \nonumber\\
& z_1 \geq 0  \nonumber\\
& z_2 \geq 0  \nonumber
\end{align}

Write a function $\texttt{LPQPa}$, with function declaration line

`def LPQPa():
    return feas, bound, zOpt, JOpt`
 

then call the function and print the outputs.

### <font color=red> Delivarable 1a: write your answer in the code cell below.</font>

In [ ]:
from pyomo.opt import SolverStatus, TerminationCondition
import pyomo.environ as pyo

# Write your code here:
def LPQPa(): 
  model = pyo.ConcreteModel()
  model.z1 = pyo.Var()
  model.z2 = pyo.Var()

  model.obj = pyo.Objective(expr = -5*model.z1 - 7*model.z2)
  model.Constraint1 = pyo.Constraint(expr = -3 * model.z1 + 2*model.z2 <= 30)
  model.Constraint2 = pyo.Constraint(expr = -2 * model.z1 + model.z2 <= 12)
  model.Constraint3 = pyo.Constraint(expr = model.z1 >= 0) 
  model.Constraint4 = pyo.Constraint(expr = model.z2 >= 0)

  solver = pyo.SolverFactory('ipopt')
  results = solver.solve(model)

  return check_solver_status(model,results)
print(LPQPa())

    model.name="unknown";
      - termination condition: unbounded
      - message from solver: Ipopt 3.12.13\x3a Iterates diverging; problem
        might be unbounded.
================ Problem is unbounded ==================
(True, False, [], inf)


$\textbf{Part (b)}$

\begin{align}
\min_{z_1,z_2}~ & 3z_1 + z_2  \\
\text{s.t. } & z_1 - z_2 \leq 1  \\
& 3z_1 + 2z_2 \leq 12  \\
& 2z_1 + 3z_2 \leq 3  \\
& -2z_1 +3z_2 \geq 9  \\
& z_1 \geq 0  \\
& z_2 \geq 0 
\end{align}

Write a function $\texttt{LPQPb}$, with function declaration line

`def LPQPb():
    return feas, bound, zOpt, JOpt`

then call the function and print the outputs.

### <font color=red> Delivarable 1b: write your answer in the code cell below.</font>

In [ ]:
# Write your code here:

$\textbf{Part (c)}$

\begin{align}
\min \hspace{2mm}&  \| \begin{bmatrix}
z_1             \\[0.3em]
z_2 +5            \\[0.3em]
\end{bmatrix}\|_1 + \|\begin{bmatrix}
z_1-2             \\[0.3em]
z_2             \\[0.3em]
\end{bmatrix}\|_\infty
\nonumber\\
\text{subject to} \hspace{12mm}& 3z_1+2z_2 \leq -3 \nonumber\\
&\hspace{6mm} 0 \leq z_1 \leq 2 \nonumber\\
&\hspace{2mm}-2 \leq z_2 \leq 3 \nonumber
\end{align}

Note: Use the LP reformulation of this problem.

Write a function $\texttt{LPQPc}$, with the following function declaration line, then call the function and print the outputs.

`def LPQPc():
    return feas, bound, zOpt, JOpt`

### <font color=red> Delivarable 1c: write your answer in the code cell below.</font>

In [ ]:
# Write your code here:

$\textbf{Part (d)}$

\begin{align}
\min_{z_1,z_2}~ & z_1^2 + z_2^2 \\
\text{s.t. } & z_1 \leq -3 \\
& z_2 \leq 4\\
& 0 \geq 4z_1+3z_2  
\end{align}

Write a function $\texttt{LPQPd}$, with the following function declaration line, then call the function and print the outputs.

`def LPQPd():
    return feas, bound, zOpt, JOpt`

### <font color=red> Delivarable 1d: write your answer in the code cell below.</font>

In [ ]:
# Write your code here:

***

# <font color=blue> 2. Nonlinear Programming I: </font>

$\textbf{Part (a)}$

Write a function $\texttt{NLP1}$, which solves the optimization problem defined below using $\texttt{pyomo}$, with function declaration line

`def NLP1(z0):
    return zOpt, JOpt`

where the input $\texttt{z0}$ is the initial guess for your optimizer. The function should have 2 outputs. $\texttt{zOpt}$ is the optimizer and $\texttt{JOpt}$ is the optimal cost. Also, call the function and print the outputs.

\begin{align}
\min_{z_1,z_2}~ & 3\sin(-2\pi z_1)+2z_1+4+\cos(2\pi z_2)+z_2 \\
\text{s.t. } &-1 \leq z_1 \leq 1 \\
&-1 \leq z_2 \leq 1
\end{align}

### <font color=red> Delivarable 2a: write your answer in the code cell below.</font>

In [ ]:
# Write your code here:
import numpy as np
import pyomo.environ as pyo
from pyomo.opt import SolverStatus, TerminationCondition

def NLP1(z0 = [1,1]):
  model = pyo.ConcreteModel()
  model.z1 = pyo.Var(initialize = z0[0])
  model.z2 = pyo.Var(initialize = z0[1])
  model.obj = pyo.Objective(expr = 3*pyo.sin(-2*np.pi*model.z1)+ 2*model.z1 + 4 + pyo.cos(2*np.pi*model.z2) + model.z2)
  model.constraint1 = pyo.Constraint(expr = (-1, model.z1, 1))
  model.constraint2 = pyo.Constraint(expr = (-1, model.z2, 1))
  # model.constraint2 = pyo.Constraint(expr = pyo.inequality(-1, model.z2, 1))  #This line and the above line are equivalent. 
  # Range inequlities can be represented using both the above syntaxes.

  solver = pyo.SolverFactory('ipopt')
  results = solver.solve(model)
  print('zOpt = ', np.array([pyo.value(model.z1), pyo.value(model.z2)]))
  print('JOpt = ', pyo.value(model.obj))
  return np.array([model.z1,model.z2,model.obj])


$\textbf{Part (b)}$

Show the outputs of your function $\texttt{NLP1}$ for 10 random initial guesses, drawing them from a uniform random distribution over $\left[-1, 1\right]$ across your feasible set.

### <font color=red> Delivarable 2b: write your answer in the code cell below.</font>

In [ ]:
# Write your code here:

$\textbf{Part (c)}$

Print out a plot of the cost function contour, mark out the initial guesses from Part (b) and the optimal solutions. Finally, by using a 3D plot show whether the obtained solutions are global or local optima. Provide printed plots as well as your code.

### <font color=red> Delivarable 2c: write your answer in the code cells below.</font>

In [ ]:
import matplotlib.pyplot as plt
# Contour plot

# Write your code here:

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
# 3D plot

# Write your code here:

***

# <font color=blue> 3. Nonlinear Programming II </font>

Using $\texttt{pyomo}$, repeat all parts of Problem 3 but with the optimization problem defined below. Write a function $\texttt{NLP2}$ with function declaration line

`def NLP2(z0):
    return zOpt, JOpt`

\begin{align}
\min_{z_1,z_2}~ & \log(1+z_1^2)-z_2 \\
\text{s.t. } &-(1+z_1^2)^2+z_2^2 = 4 
\end{align}

$\textbf{Part (a)}$

Repeat part (a) of problem 2.

### <font color=red> Delivarable 3a: write your answer in the code cell below.</font>

In [ ]:
import numpy as np
import pyomo.environ as pyo

# Write your code here:

$\textbf{Part (b)}$

Repeat part (b) of problem 2.

### <font color=red> Delivarable 3b: write your answer in the code cell below.</font>

In [ ]:
# Write your code here:

$\textbf{Part (c)}$

Repeat part (c) of problem 2, but draw only $z_1$ from a uniform random distribution over $\left[-1, 1\right]$.

### <font color=red> Delivarable 3c: write your answer in the code cells below.</font>

In [ ]:
import matplotlib.pyplot as plt
# Contour plot

# Write your code here:

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
# 3D plot

# Write your code here:

***

# <font color=blue> 4. Mixed-integer Programming </font>

Use $\texttt{pyomo}$ to solve the two following optimization problems. Write individual functions for each optimization problem.

$\textbf{Part (a)}$

\begin{align}
\min_{z_1,z_2}~ & -6z_1-5z_2 \\
\text{s.t. } & z_1+4z_2 \leq 16 \\
& 6z_1+4z_2 \leq 28\\
& 2z_1-5z_2 \leq 6\\
& 0 \leq z_1 \leq 10\\
& 0 \leq z_2 \leq 10 \\
& z_1, z_2 \in \textbf{Z},\text{(integer)}
\end{align}

Write a function $\texttt{MIPa}$, with function declaration line

`def MIPa():
    return zOpt, JOpt`

then call the function and print the outputs.

$\textit{Hint:}$ Use $\texttt{pyo.Integers}$ to define integer decision variables.

### <font color=red> Delivarable 4a: write your answer in the code cell below.</font>

In [ ]:
# Write your code here:

$\textbf{Part (b)}$

\begin{align}
\min_{z_1,z_2}~ & -z_1-2z_2 \\
\text{s.t. } & \text{either } 3z_1+4z_2 \leq 12~~ \text{or } 4z_1+3z_2 \leq 12\\
& z_1\geq0\\
& z_2\geq0
\end{align}

Write a function $\texttt{MIPb}$, with function declaration line

`def MIPb():
    return zOpt, JOpt`

then call the function and print the outputs.

$\textit{Hint:}$ Use $\texttt{pyo.Var(within=pyo.Binary)}$ to define integer decision variables in $\texttt{Pyomo}$.

### <font color=red> Delivarable 4b: write your answer in the code cell below.</font>

In [ ]:
# Write your code here:

***

# <font color = blue> 5. KKT Conditions I </font>

The following problems also include some of the examples from Homework 2. Write functions that return a single logical variable that reflects whether all KKT conditions are satisfied for the constrained problem (i.e., 1 stands for all KKT conditions satisfied). Please submit your solutions as individual functions for each of the 4 parts. 

$\textbf{Part (a)}$

\begin{align}
\min_{z_1,z_2}~ &  -5z_1 -7z_2  \nonumber\\
\text{s.t. } & -3z_1 +2z_2 \leq 30  \nonumber\\
& -2z_1 + z_2 \leq 12  \nonumber\\
& z_1 \geq 0  \nonumber\\
& z_2 \geq 0  \nonumber
\end{align}

Write a function $\texttt{LPQPkkta}$, with function declaration line

`def LPQPkkta():
    return KKTsat`

then call the function and print the output.

### <font color=red> Delivarable 5a: write your answer in the code cell below.</font>

In [ ]:
# Write your code here:

$\textbf{Part (b)}$

\begin{align}
\min_{z_1,z_2}~ & 3z_1 + z_2  \\
\text{s.t. } & z_1 - z_2 \leq 1  \\
& 3z_1 + 2z_2 \leq 12  \\
& 2z_1 + 3z_2 \leq 3  \\
& -2z_1 +3z_2 \geq 9  \\
& z_1 \geq 0  \\
& z_2 \geq 0 
\end{align}

Write a function $\texttt{LPQPkktb}$, with function declaration line

`def LPQPkktb():
    return KKTsat`

then call the function and print the outputs.

### <font color=red> Delivarable 5b: write your answer in the code cell below.</font>

In [ ]:
# Write your code here:

$\textbf{Part (c)}$

\begin{align}
\min_{z_1,z_2}~ & z_1^2 + z_2^2 \nonumber\\
\text{s.t. } & z_1 \leq -3 \nonumber\\
& z_2 \leq 4 \nonumber\\
& 0 \geq 4z_1+3z_2  \nonumber
\end{align}

Write a function $\texttt{LPQPkktc}$, with function declaration line

`def LPQPkktc():
    return KKTsat`

then call the function and print the outputs.

### <font color=red> Delivarable 5c: write your answer in the code cell below.</font>

In [ ]:
# Write your code here:

***